# LAUNCHPAD WEB API SUMMARY
---
## Introduction
1. the **Launchpad** website lets you learn about and manage bugs, projects, questions, and other artefacts of software development. 
2. these same objects are gradually being exposed through a **web service**, so that you can access them from scripts, applications, or other websites, in addition to accessing them through the **Launchpad** website. 
3. the service is designed around the principles of **REST**, with the goals of simplicity and transparency. 

## Launchpad Resources
1. every object in **Launchpad** (*i.e. everything you might think of as having a separate identity*) has **its own URL** in the **Launchpad web service**. 
2. you can bookmark this URL and pass it around, but you can also manipulate the underlying **Launchpad object by making HTTP requests to its URL**.

> (This is the plan, anyway. We're still working to expose all of Launchpad's objects through the web service, and many objects that are exposed don't yet publish much useful information.)

> To make authenticated requests, you'll need to digitally sign those requests using a set of OAuth credentials.
Unauthenticated requests can see all public objects, which are the majority, but they can't see private objects (such as hidden email addresses, security-sensitive bugs, or private teams), they can't change anything, and of course they have no concept of a "me" resource to start from. Trying to access a private resource will give you a "401 Unauthenticated" error.

### The homepage
1. **Launchpad**'s website:
    - has a **homepage** that acts as a jumping-off point to projects, bugs, people, answers, and so on. 
    - contains links to other parts of Launchpad
2. **Launchpad**'s web service has a **homepage**, too:
    - a lot more sparse than the web site's homepage but it also makes a good jumping-off point.
    - serves the same purpose as the launchpad's website homepage: it tells you about the major parts of the **Launchpad** web service.
    - can be obtained by an **HTTP GET request** to the **base URL** of the web service (*in development mode*): http://api.launchpad.net/devel/.
    - returns a  **JSON hash document** response (*i.e. JSON object with unicode values*) when it's queried, which you can turn into a native-language data structure using whatever libraries are available for the programming language you're using.
    - contains three fields pointing to links, which can be followed to explore the web service:
        1. `people_collection_link`: **list of people tracked by Launchpad**.
        2. `bugs_collection_link`: **list of bugs tracked by Launchpad**.
        3. `me_link`: **link to your user account**. (*if signed in with an account*)
3. **notices**: 
    - almost all of the documents served through the **Launchpad** web service are JSON documents. In fact, **almost all of them describe dictionaries of key-value pairs**.
    - in `launchpadlib`, we use the Python `simplejson` library to process JSON documents. So we can use this library to process the response and obtain the JSON hash document.

Apart from **the homepage**, there are two basic types of objects in launchpad: **entries** and **collections**:
1. `entry`: a **single object**, like a bug or your user account;
2. `collection`: a **group of entries**.
3. *by convention*, all fields that are **links** to other parts of the **Launchpad** web service have **names** ending in `_link`, for example: 
    - **links to collections**: always have names that end in `_collection_link`; 
    - **links to entries**: always have names that just end in `_link`. 
    
There's also a fourth type of Launchpad resources: **hosted files**: 
1. this resource is a **front-end to a file stored in Launchpad's file library**. 
2. for example, a person's mugshot image: you can find the URL to this resource by looking under `mugshot_link` in the JSON representation of a `person`. It should look like "/1.0/~{person}/mugshot".

### Entries
1. An **entry-type resource** responds to a specific HTTP interface (that's common to all entry resources exposed by the Launchpad web service): **GET**, **PUT**, and **PATCH**. 
2. **Entry resources** may also respond to **custom named GET and POST operations which are different for every kind of entry**.

#### Fields
There are **three and only three kinds of fields**:
1. **Atomic chunks of data**. Examples here include `date_created`, `display_name`, and `time_zone`. These may be of any JSON data type. Some of these can be modified: you can change your own `display_name`, but you can't change `date_created`. (See "**WADL Description**" for more information)
2. **Links to other entry-type resources**. For example, `mugshot_link` points to your mugshot image. `preferred_email_address_link` points to a resource that represents your preferred email address. Some of these links are modifiable (See "**WADL Description**" for more information). Two of these links are especially important, and you'll find them present in every representation of an entry-type resource:
    - `self_link` is the **URL** to the **resource itself**.
    - `resource_type_link` is a **link** to a **machine-readable description of this resource**. You can use this to do introspection on the resource, finding out what special operations are available, or which of the fields in the representation can be modified. (See "**WADL Description**" for more information) 
3. **Links to collection-type resources**. A `person` in Launchpad can be associated with more than one email address, but only one of those can be the '**preferred**' address at any one time. The `preferred_email_address_link` field points to whatever address is currently preferred. The `confirmed_email_addresses_collection_link` field points to a list containing all the addresses. You can never change a link to a collection.

### Collections
The other main sort of resource in the **Launchpad** web service is a **collection-type resource**: 
1. a container for a number of other resources. 
2. as with entry resources, every container resource:
    - has its own URL that you can bookmark or pass around.
    - return a JSON document describing the collection upon receiving a GET request

All collection resources serve JSON documents, whether they're collections of bugs, people, bug tasks, email addresses languages, or whatever. It's always a JSON hash with keys called `total_size`, `resource_type_link`, and `entries`:
1. `total_size`: the number of items in the collection ; 
2. `resource_type_link`: a machine-readable description of the collection (see "**WADL Description**" for more information) ;
3. `entries`: the actual entries of the collection. 

A request issued at a collection won't send back *all* the **entries** it contains:
1. for example, putting over 250,000 bugs in one document would be crazy. **Launchpad**'s web service does the same thing as the **Launchpad website**: 
    - it sends you **one page of entries at a time**, and **includes links** (*where appropriate*) to the **next and previous pages**. 
    - so `entries` is a **JSON list containing 75 JSON hashes, each describing one entity-type resource**. Each hash contains the same information as if you'd sent a GET request to that entity's `self_link`.
    - for more than 75 entities, send a GET request to the `next_collection_link`. 
    - for some other number of entities, starting from entity 20 in the list instead of the first for example, you can manually vary the `ws.start` and `ws.size` **URL parameters**:
        1. sending a GET request to http://api.staging.launchpad.net/1.0/bugs?ws.start=9&ws.size=3 would get you three bugs: the ones that would be accessible from "collection['entries'][9:12]" if you'd sent GET to http://api.staging.launchpad.net/1.0/bugs and retrieved the first 75.

**Notice**  
For consistency's sake, **all collection resources** serve JSON hashes with `total_size` and the rest, even collections which are very unlikely to have more than 75 entries, *like someone's list of spoken languages*.

### Named operations
1. **named operations** are custom operations available on specific resources, **identified by name rather than by the name of one of the standard HTTP methods**. 
2. they're different for every kind of resource (See reference documentation "**WADL Description**" for more information). 
3. A **named operation** either **modifies** the **Launchpad** data set or it doesn't:
    - If it's **read-only**, then you access it with HTTP GET
    - If it's a **write operation**, you need to access it with HTTP POST.
4. In general, you **invoke a named operation on a resource** by tacking the query parameter "`ws.op={operation name}`" onto the resource's URL. It's just like calling a method in a programming language: 
    - the resource is the object and the operation is the method. 
    - Any arguments to the method are appended as additional query parameters.
5. **notice**: When passing **parameters** to **methods**, it is necessary to be aware of `lazr.restful`'s **data type marshalling** (*basically, data is represented as if it were fragments of JSON*), which means **string parameters need quotes around them**. 

#### Read operation (GET)

Example: To invoke the person search operation, we make a GET request to http://api.staging.launchpad.net/1.0/people?ws.op=find&text={text} where "{text}" is the text you want to search for.

The response to a read operation can be any JSON document, but it's usually a JSON hash that looks exactly like the JSON representation of a collection resource (It's got `total_size`, `entries`, possibly `next_collection_link`, and so on).

### Hosted files
#### Read operation (GET)
When you send a GET request to a hosted file resource, you'll get back an HTTP redirect to a file in **Launchpad**'s file library.

    GET /1.0/~salgado/mugshot HTTP/1.1
    Host: api.staging.launchpad.net

You'll get back a response that looks like this:

    303 See Other
    Location: Location: http://staging.launchpad.net:58000/92/mugshot

Send a second GET request to the URL in Location, and you'll get the image document itself.

    GET /92/mugshot HTTP/1.1
    Host: staging.launchpad.net:58000

    200 OK
    Content-Type: image/jpeg

    [image goes here]

## Compressing responses
1. **Launchpad**'s web service serves XML and JSON documents that can be compressed prior to being sent over the network, by specifying a compression algorithm in the "**TE**" request header $\rightarrow$ get them faster and save bandwidth
2. **Launchpad**'s web service supports **two compression algorithms**, that are both defined in the HTTP standard : 
    - "**gzip**", the standard gzip algorithm handled by Python's `gzip` module ;
    - "**deflate**", the algorithm handled by Python's `zlib` module.

The **TE header**:
1. `TE: deflate` or
2. `TE: gzip`

**Launchpad** will send compressed data, and will set the **Transfer-Encoding response header** to the name of the compression algorithm it used. It'll either look like:
1. `Transfer-Encoding: deflate` or
2. `Transfer-Encoding: gzip`

## Caching responses
1. **principle**: cache documents you get from **Launchpad**, especially documents like the **WADL file** that are **large** and don't change very often.
2. process (used through `httplib2`):
    - when **Launchpad** makes a GET request and gets a document back, it stores the document in a file, along with all of the HTTP response headers. 
    - the next time it needs to make that GET request, it uses the cached response instead of making another request to get the same data back.
3. to a check if a document hasn't changed since the last time it was retrieved, without making the same request again:
    - when you get a document from **Launchpad**, you'll also get a value for the **HTTP response header** "**ETag**": `ETag: "924eb0c15c911d64e633b5f012d046d04a83b571"`
    - If you suspect the document **has changed**, make a **GET request** to the **document's URL**, but include the **ETag** in the **HTTP header** "**If-None-Match**": `If-None-Match: "924eb0c15c911d64e633b5f012d046d04a83b571"`
    - If the document **has changed**, this will work just like a **normal GET request**. You'll get back the **changed version of the document**, including **a new ETag**. 
    - If the document is the **same** as it used to be, you'll get an **HTTP response** `304 Not Modified` :
        1. Instead of sending the document again, **Launchpad** is telling you that you already have the most recent version. 
        2. This is called "**conditional GET**".
4. **notice**: **Launchpad** doesn't serve **ETags** for **collections**, only for **individual objects** and for the **server root**. 

## The reference documentation
The reference documentation will tell you about all the fields in an object's JSON representation, and all the HTTP methods it will respond to: https://launchpad.net/+apidoc/devel.html

## The WADL Description
1. **definition**: 
    - a **machine-readable document** containing information about the **Launchpad**'s resources in **WADL format**.
    - Almost every interesting aspect of the web service is described in this document.
2. **utility**:  build tools interacting with the web services, that are loosely coupled to its design.
3. **content**:
    - "**Default representation**" **section**: available **attributes**.
    - "**Custom POST methods**" and "**Custom GET methods**" **sections**: 
        1. **methods** the object supports other than the **default methods**. 
        2. The **methods** take whatever **parameters** are listed in "**Request query parameters**". 
        3. **notice**: You can ignore the "**ws.op**" **parameter** because you're using `launchpadlib`; that's just the **name of the method**. 
4. **location** = **the service root**: https://api.staging.launchpad.net/devel/
5. by **Launchpad convention**, every **entry resource JSON hash document** has a `resource_type_link` that's an **index** into this **document**, for example: 
    - "http://api.staging.launchpad.net/1.0/#person" is a reference to the **XML tag** in **this document** with the **ID** `person`. 
    - **XML tag** : describing the capabilities of a `person` entry resource.
. **notices**:
    - **reference documentation**: a human-readable transformation of the WADL document. 
    - `launchpadlib`: a thin wrapper on top of a generic **WADL** library: it becomes a **Launchpad** library when it reads in **Launchpad**'s **WADL file**.
5. GET request to a **resource** in the WADL document: request a WADL representation instead of the JSON one

        GET /1.0/~my-user-account HTTP/1.1
        Host: api.staging.launchpad.net
        Accept: application/vd.sun.wadl+xml

> What's not defined in this file? Mainly, there's a lack of information about our URL structure. You've already seen that you can get a description of any person in **Launchpad** by sending GET to http://api.staging.launchpad.net/1.0/~{name} and plugging in the name. This is a useful shortcut that can often save you a few HTTP requests, but the WADL file doesn't say anything about that. It's possible to put this information into WADL; we just haven't implemented it yet.

6. **GET response** : a **small WADL document** that contains a **reference** to the **large WADL document** at the **service root** $\rightarrow$ useful:
    - if lost and need to get back on track ;
    - if you don't want to rely on the **Launchpad-specific** `resource_type_link` convention.
    
## `launchpadlib`
### Introduction
1. `launchpadlib` is the **official Python client library for Launchpad's web service**. 
1. it is an **open-source Python library** that lets you treat the **HTTP resources** published by **Launchpad's web service as Python objects responding to a standard set of commands**. 
2. with `launchpadlib` you can integrate your applications into **Launchpad** without knowing a lot about HTTP client programming. 
3. `launchpadlib` is based on `httplib2`, a small, fast HTTP client library for Python.
4. **HTTP requests and responses go back and forth behind the scenes**.

There are many situations where you wouldn't use `launchpadlib`: 
1. if you're not a Python programmer ;
2. if you want to write an Ajax client that runs in a web browser ;
3. if `launchpadlib` is too heavyweight for what you want to do ;
4. if you just want to understand what's going on between the client and the server.

> Launchpad's web service currently exposes the following major parts of Launchpad:
    1. People and teams
    2. Team memberships
    3. Bugs and bugtasks
    4. The project registry
    5. Hosted files, such as bug attachments and mugshots. 
> As new features and capabilities are added to the web service, you'll be able to access most of them without having to update your copy of launchpadlib. You will have to upgrade launchpadlib to get new client-side features (like support for uploaded files). The Launchpad team will put out an announcement whenever a server-side change means you should upgrade launchpadlib. 

### Installation
```bash
sudo apt-get install python-launchpadlib #ubuntu apt installation
pip install launchpadlib #pip installation
```

### Versions
> The Launchpad webservice is published in versions. We support released applications, such as apport, while continuing to improve our API in backwards incompatible ways. Versions will remain supported until the Ubuntu release that uses it becomes unsupported.  
> **NOTE**: We have the mechanism to have separate, frozen versions. We need to actually have tests showing that APIs supporting key applications do not change. We may need to get community help for this.  
> See this page for the available versions: https://edge.launchpad.net/+apidoc/

### Getting started
1. choose a **cache directory**.
2. set up **credentials**:
    - **read-only** access to Launchpad data: anonymous access ;
    - **read/write** acces to Launchpad data: authentication with `OAuth`.

```python
cachedir = "/home/me/.launchpadlib/cache/" #choose a cache directory

from launchpadlib.launchpad import Launchpad

"""
automatic negotiation for a credential to have a read-only access to public Launchpad data
returns a launchpad http client object with an anonymous access to Launchpad's web API

- first argument: string that identifies the web service client.
- second argument: Launchpad instance to run against. Here, we're using "production", which is mapped to the web service root on the production Launchpad server: "https://api.launchpad.net/".

- version: API version to use. For historical reasons the default is '1.0', and you may want to use this in certain special circumstances, but in most cases you should use 'devel' so that you get a reasonably complete and current interface. 

error we try to modify the dataset, access private data, or access objects like launchpad.me which assume a particular user is logged in. 

only available in launchpadlib starting in version 1.5.4

"""
launchpad = Launchpad.login_anonymously('just testing', 'production', cachedir, version='devel')
bug_one = launchpad.bugs[1] # we can use the Launchpad object right away.
print(bug_one.title) # Microsoft has a majority market share
```

### Getting help
```python
print(bug_one.lp_attributes); # Data fields of the resource object. All of them are readable, some of them are writable.
print(bug_one.lp_collections); # List of launchpad collection objects associated with the object.
print(bug_one.lp_entries); # List of Launchpad entries objects associated with the object.
print(bug_one.lp_attributes); # List of the names of methods that can be invoked on the object.
print(repr(bug_one)); # <bug at https://api.staging.launchpad.net/beta/bugs/1>

# https://launchpad.net/+apidoc#bug : to get the WADL description of the bug object
```

> the reference documentation still needs some work, and it's geared more towards web service hackers than launchpadlib users, but it will tell you about all of this object's attributes and all the supported operations. 

### Top level objects
The **Launchpad** object has **attributes** corresponding to the **major parts of Launchpad**:
1. `bugs`: All the bugs in Launchpad
2. `people`: All the people in Launchpad
3. `me`: You
4. `distributions`: All the distributions in Launchpad
5. `projects`: All the projects in Launchpad
6. `project_groups`: All the project groups in Launchpad 

**Notice**: `distributions`, `projects` and `project_groups` are all actually the same thing.
**Examples**:
```python
    me = launchpad.me
    print(me.name)
    # This should be your user name, e.g. 'salgado'
    
    people = launchpad.people # other people who use Launchpad
    salgado = people['salgado'] # person entry object with name = salgado
    print(salgado.display_name) # display name of the object
    # Guilherme Salgado
    
    # another way of finding a person entry object
    salgado = people.getByEmail(email="guilherme.salgado@canonical.com") # email is a keyword argument
    print(salgado.display_name)
    # Guilherme Salgado
    
    # searches returning more than one result
    for person in people.find(text="salgado"):
        print(person.name)
    # agustin-salgado
    # ariel-salgado
    # axel-salgado
    # bruno-salgado
    # camilosalgado
    # ...
```

### Entries
1. **definition**: an object representing single piece of data representing a resource in Launchpad published through its web service, for example ***bugs, people, projects, team memberships, $\dots$***
2. **attributes**:
    - role: **facts about the entry**.
    - `self_link`: **permanent ID** of the **entry object**.
    - `web_link`: **Launchpad's website URL** of the **entry**, if it has one.
    - some of them : 
        1. are links to other **entries** with attributes of their own.
        2. are links to related **collections**.
        3. designate **named operations** with **keyword arguments only** (*no positional arguments*). 
3. **example**:

```python
    # name and display_name are facts about a person entry
    print(salgado.name)
    # salgado

    print(salgado.display_name)
    # Guilherme Salgado
    
    # private and description are facts about a bug
    print(bug_one.private)
    # False

    print(bug_one.description)
    # Microsoft has a majority market share in the new desktop PC marketplace.
    # This is a bug, which Ubuntu is designed to fix.
    # ...
    
    # an entry attribute of the bug object, corresponding to its owner object
    owner = bug_one.owner
    print(repr(owner))
    # <person at https://api.staging.launchpad.net/beta/~sabdfl>
    print(owner.name)
    # sabdfl
    print(owner.display_name)
    # Mark Shuttleworth
```

### Errors
1. **principle**: when the Launchpad web service encounters an **error**, it sends back an **error message** to `launchpadlib`, which raises an `HTTPError` **exception**. 
2. **result**: **information** about the **HTTP request** that caused the **error**, and the **server-side error message**.
3. **consequence**: depending on the error, we may be able to **recover** or **change** our code and try again. 

### Collections
1. **definition**: an **object** representing **similar entries grouped together**.
2. **properties**:
    - **iterable**: `for entry in collection: \ do stuff`
    - **slicable**:
        1. `collection[0:positive_number:size]`: we can only slice from the beginning of a collection up to a definite and positive number, while optionally specifying the slice's size (by default, size = 1)
        2. not to iterate over all contents of a collection, which can contain a huge number of entries $\rightarrow$ `launchpadlib` will just keep pulling down entries until it runs out, which might be forever (or, realistically, until the client is banned for making too many requests). 
3. **examples**:

```python
    # iterating over a collection of person entries
    for person in launchpad.people.find(text="salgado"):
        print(person.name)
    
    # accessing a bug object's associated bug tasks collection
    tasks = bug_one.bug_tasks
    
    print(len(tasks))
    # 17
    for task in tasks:
        print(task.bug_target_display_name)
    # Computer Science Ubuntu
    # Ichthux
    # JAK LINUX
    # ...
    
    # accessing a person's associated languages collection
    for language in salgado.languages:
        print(language.self_link)
    # https://api.staging.launchpad.net/beta/+languages/en
    # https://api.staging.launchpad.net/beta/+languages/pt_BR
    
    # accessing the 10 most recently filed bugs
    bugs = launchpad.bugs[:10]
    for bug in bugs:
        print(bug.description)

    # Possible errors
    launchpad.bugs[-5:]
    # *** ValueError: Collection slices must have a nonnegative start point.

    bugs[10:]
    # *** ValueError: Collection slices must have a definite, nonnegative end point.

    bugs[:-5]
    # *** ValueError: Collection slices must have a definite, nonnegative end point.

    # for a slice of 10 bugs, access every other bug (i.e. alternate between including and skipping a bug)
    every_other_bug = launchpad.bugs[0:10:2]
    len(every_other_bug)
    # 5
```
### Hosted files
1. **definition**: data in binary files form stored by Launchpad.
2. **example**: people's mugshots.
3. **utility**: read/write these binary files programmatically
4. **approach**:
    - getting a `launchpadlib` **reference** to one of these **hosted files**.
    - **reading data** from host file object by invoking the `open()` method on it $\rightarrow$ **result** = **filehandle** (*i.e. classic I/O operations*).
    - getting an **error** if the **file doesn't exist** (*e.g. if a person entry doesn't have a mughost*).
5. example:

```python
    mugshot = launchpad.me.mugshot # getting mugshot of current user
    mugshot_handle = mugshot.open() # opening the mugshot host file of current user
    mugshot_handle.read() # reading data from host file object
    # [binary data]
    mugshot_handle.content_type
    # 'image/jpeg'
    mugshot_handle.last_modified
    # 'Wed, 12 Mar 2008 21:47:05 GMT'
    
    launchpad.people['has-no-mugshot'].mugshot
    # *** HTTPError: HTTP Error 404: Not Found

```

### Persistent references to Launchpad objects
1. **principle**:
    - every **entry** and **collection** has a **unique ID**: its **URL**. 
    - get this unique ID by **invoking** `str()` on the **object**.
2. **utility**:
    - used to **keep track** of **Launchpad objects over time**, or **pass references** to **Launchpad objects** to other **programs**
    - turn the **reference** into its **corresponding Launchpad object** by using the **lauchpad client object** `load()` **method**.
3. **example**:

```python
    # unique ID of the bug entry object
    print(str(bug_one))
    # https://api.staging.launchpad.net/beta/bugs/1
    
    # loading the Launchpad object corresponding to the unique ID passed to the load method
    bug_one = launchpad.load("https://api.staging.launchpad.net/beta/bugs/1")
    print(bug_one.title)
    # Microsoft has a majority market share
```

4. viewing JSON objects of from **Launchpad** objects in a browser:
    - `launchpadlib` provides a nice Python wrapper around JSON objects, but it does allow you to directly access the JSON itself. 
    - `object.self_link;`: to view a **JSON file** in a browser. 

### To have a faster client
1. have the lastest version of `launchpadlib`:
    - (1.10.6 at 09/05/2019)
    - 1.5.1 or above is acceptable.
2. fetch objects only once:

```python

# Don't do this
if bug.person is not None:
    print(bug.person.name)

# Do this
p = bug.person
    if p is not None:
        print(p.name)
```